In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 2000
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.
W0913 01:20:12.662409 11164 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0913 01:20:12.674412 11164 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0913 01:20:12.675412 11164 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0913 01:20:12.696417 11164 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0913 01:20:12.698417 11164 deprecation_wrapper.py:11

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 4s 67us/step - loss: 1.1308 - acc: 0.6494 - val_loss: 0.4495 - val_acc: 0.8563
Test loss: 0.4494672349214554
Test accuracy: 0.8563


In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

## data[2].astype(dtype=float32)

In [3]:
import numpy as np

for i in range(0,7,2):
    print(str(i+1) + '. ' + str(model.get_weights()[i].shape) + ', ' + str(i+2) + '. '+str(model.get_weights()[i+1].shape))

1. (3, 3, 1, 32), 2. (32,)
3. (3, 3, 32, 64), 4. (64,)
5. (9216, 128), 6. (128,)
7. (128, 10), 8. (10,)


In [23]:
def gen_normal_noise(weights_shape, scaling='normalized'):
    noise = np.random.normal(size=weights_shape)
    if scaling == 'normalized':
        noise =2*noise/(np.max(noise))
    else:
        noise = noise/2
    return noise

def generate_noisy_model(keras_model):
    len_layers = len(keras_model.layers)
    
    for i in range(len_layers):
        len_list_weights = len(keras_model.layers[i].get_weights())
        weight_list_dummy = []
        
        for j in range(len_list_weights):
            if keras_model.layers[i].get_weights()[j].size > 0:
                weights_shape = keras_model.layers[i].get_weights()[j].shape
                noise = gen_normal_noise(weights_shape)
                
                weight_list_dummy.append( keras_model.layers[i].get_weights()[j] + noise)
        keras_model.layers[i].set_weights(weight_list_dummy)
        del weight_list_dummy
        return keras_model
    
def generate_population(keras_model, no_of_individuals):

    accuracies = []
    models = []
    
    for population in range(no_of_individuals):
        new_model = generate_noisy_model(keras_model)
        acc_dummy = new_model.evaluate(x_train, y_train, verbose=0)[1]
        models.append(new_model)
        accuracies.append(acc_dummy)
        print('generating '+ str (population) + ' individual; acc: ' + str(acc_dummy))        
        
    return models, accuracies

#findx incedes of max values in list
def n_max_values(no_list, n):
    idx =[]
    dummy_no_list = no_list.copy()
    for i in range(n):
        idx.append(np.argmax( np.asarray(dummy_no_list) ))
        no_list.pop(idx[i])
        
    return idx

#for one dimensional number list
def softmax(no_list):
    L = -np.asarray(no_list)
    denominator = np.sum( np.exp(L))
    numerator = np.exp(L)
    return numerator/denominator

def select_fit_individuals(models, accuracies, no_to_select):
    idx = n_max_values(accuracies, no_to_select)
    selected_models = [models[i] for i in idx]
    selected_acc = [accuracies[i] for i in idx]

#     del models, accuracies
    acc_prob = softmax(selected_acc)
    return selected_models, acc_prob, selected_acc
                         
def generate_next_generation(selected_models, acc_prob, generation_population):
    models_new_populations = []
    
    for i in range(len(acc_prob)):
        models_new_populations.append(int(np.round(acc_prob[i]*generation_population)))
        
    new_models = []  
    accuracies = []
    
    for i in range(len(models_new_populations)):
        models, accuracy_s = generate_population(selected_models[i], models_new_populations[i])
        print(accuracy_s)
        new_models.extend(models)
        accuracies.extend(accuracy_s)
        
    return new_models, accuracies
                         
def is_accurate_enough(accuracies, thresh):
    idx = np.argmax( np.asarray(accuracies) )
    accurate_enough = accuracies[idx] >= thresh
    return accurate_enough, idx

In [24]:
epochs = 10
gen_population = 3
no_to_select = 1

models, accuracies = generate_population(model, gen_population)
for i in range(epochs):
    selected_models, acc_prob, selected_acc = select_fit_individuals(models, accuracies, no_to_select)
    del accuracies
    models, accuracies = generate_next_generation(selected_models, acc_prob, gen_population)
    accurate_enough, idx = is_accurate_enough(accuracies, .6)
    if accurate_enough:
        break
    

generating 0 individual; acc: 0.5031833333333333
generating 1 individual; acc: 0.5613833333333333
generating 2 individual; acc: 0.5564833333333333
generating 0 individual; acc: 0.4523
generating 1 individual; acc: 0.3713666666666667
generating 2 individual; acc: 0.3268666666666667
[0.4523, 0.3713666666666667, 0.3268666666666667]
generating 0 individual; acc: 0.4345333333333333
generating 1 individual; acc: 0.44931666666666664
generating 2 individual; acc: 0.44216666666666665
[0.4345333333333333, 0.44931666666666664, 0.44216666666666665]
generating 0 individual; acc: 0.4614
generating 1 individual; acc: 0.49966666666666665
generating 2 individual; acc: 0.44055
[0.4614, 0.49966666666666665, 0.44055]
generating 0 individual; acc: 0.49966666666666665
generating 1 individual; acc: 0.44693333333333335
generating 2 individual; acc: 0.41441666666666666
[0.49966666666666665, 0.44693333333333335, 0.41441666666666666]
generating 0 individual; acc: 0.4366833333333333
generating 1 individual; acc: 

ValueError: attempt to get argmax of an empty sequence

In [22]:
accuracies

[]

In [ ]:
layer_shape

In [ ]:
model.layers[1].get_weights()[0][3] == model.layers[1].get_weights()[1]

In [ ]:
model.layers[1].get_weights()[1] == model.layers[1].get_weights()[0][0][0][0] 

In [ ]:
model.metrics_names